In [213]:
import pandas as pd
import numpy as np

data = pd.read_csv('./備捨棄 - 工作表1.csv')

In [214]:
data = data.drop(columns=['每模張數'])
data.columns=['組合', '組合占比', '卡片名稱', '卡片種類', '卡片總量']
data['組合'] = ['A', 'A', 'A', 'B', 'B', 'C', 'C']
data

,組合,組合占比,卡片名稱,卡片種類,卡片總量
0,A,60%,卡片N,6,180000
1,A,35%,卡片R,6,105000
2,A,5%,卡片SR,6,15000
3,B,22.80%,事件卡A,15,91200
4,B,77.20%,事件卡B,19,308800
5,C,50%,勝利條件卡N,6,150000
6,C,50%,勝利條件卡SP,10,150000


# A組牌  配率
>每包內含張數：3  
>同一包卡片不重複，SR最多1張，R最多2張  
>**SR為閃卡，獨立一版**

In [215]:
data_A = data[data['組合'] == 'A']
data_A.sort_values(by='卡片總量')

,組合,組合占比,卡片名稱,卡片種類,卡片總量
2,A,5%,卡片SR,6,15000
1,A,35%,卡片R,6,105000
0,A,60%,卡片N,6,180000


In [216]:
'''
Pass this part
'''

'\nPass this part\n'

# B組牌 配率
>每包內含張數：4  
>同一包卡片不重複，A最多1張	

In [217]:
data_B = data[data['組合'] == 'B']
data_B = data_B.reset_index(drop=True)
data_B

,組合,組合占比,卡片名稱,卡片種類,卡片總量
0,B,22.80%,事件卡A,15,91200
1,B,77.20%,事件卡B,19,308800


In [218]:
total = data_B['卡片總量'].sum()
total

400000

#### 占比分配

In [226]:
'''自變數'''
one_set = 4 #一套卡數
one_sect = int(81/one_set) #一版套數
sect = 3 #總版數


'''應變數'''
rounds = total / (one_set * one_sect * sect) #車數

if rounds %100 != 0:
    rounds = (rounds//100+1)*100
else:
    pass
rounds

1700.0

In [227]:
true_total

408000.0

In [361]:
true_total = one_sect * sect * one_set * rounds

a = (one_sect * sect // data_B['卡片種類'][0])-1 #手動微調
b = (one_sect * sect % data_B['卡片種類'][0])+10 #手動微調
prob = (a*15+b)*rounds/true_total

print('A卡落版',a,'組+',b,'款')
print('A卡實際機率：{:.4%}\n機率差值：{:.4%}'.format(prob, (prob-0.2280)))
if b != 0:
    print('款式落差百分比為：{:.4%}'.format( (1)*rounds/true_total) )
else:
    print('款式落差百分比為：{:.4%}'.format( (0)*rounds/true_total) )

A卡落版 3 組+ 10 款
A卡實際機率：22.9167%
機率差值：0.1167%
款式落差百分比為：0.4167%


In [362]:
# true_total = one_sect * sect * one_set * rounds

c = (((one_sect * sect * one_set) - (a*15+b)) // data_B['卡片種類'][1])
d = (((one_sect * sect * one_set) - (a*15+b))  % data_B['卡片種類'][1])
prob = (c*19+d)*rounds/true_total

print('B卡落版',c,'組+',d,'款')
print('B卡實際機率：{:.4%}\n機率差值：{:.4%}'.format(prob, (prob-0.772)))
if b != 0:
    print('款式落差百分比為：{:.4%}'.format( (1)*rounds/true_total) )
else:
    print('款式落差百分比為：{:.4%}'.format( (0)*rounds/true_total) )

B卡落版 9 組+ 14 款
B卡實際機率：77.0833%
機率差值：-0.1167%
款式落差百分比為：0.4167%


#### 卡片分配

In [372]:
import random

#A卡池
while True:
    A_base = []
    for i in range(data_B['卡片種類'][0]):
        A_base.append('A'+str(i+1))
    A = A_base*3
    random.shuffle(A_base)
    for i in range(15-10):
        A_base.pop()
    A_base.sort(reverse=True)
    A.extend(A_base)
    
    if A_base[-1] != 'A1':
        break
    else:
        continue

A_base

['A8', 'A7', 'A6', 'A5', 'A4', 'A3', 'A15', 'A13', 'A12', 'A10']

In [373]:
#B卡池
while True:
    B_base = []
    for i in range(data_B['卡片種類'][1]):
        B_base.append('B'+str(i+1))
    B = B_base*9
    random.shuffle(B_base)
    for i in range(19-14):
        B_base.pop()
    B_base.sort(reverse=True)
    B.extend(B_base)

    if B_base[-1] != 'B1':
        break
    else:
        continue

B_base

['B8',
 'B7',
 'B6',
 'B5',
 'B4',
 'B3',
 'B2',
 'B19',
 'B17',
 'B16',
 'B15',
 'B13',
 'B12',
 'B10']

In [374]:
tt = [len(A), len(B)]
tt

[55, 185]

In [375]:
submit = []

while A != []: #配選ABBB牌
    comb = []
    comb.append(A.pop())
    while len(comb)<4: #隨機組合不重複
        B_chosen = random.choice(B)
        if B_chosen not in comb:
            comb.append(B_chosen)
            B.remove(B_chosen)
        else:
            continue
    comb.sort()
    if comb not in submit: #退還已存在組合
        submit.append(comb)
    else:
        for i in range(3):
            B.append(comb.pop())
        A.append(comb.pop())

In [376]:
tt = [len(A), len(B)]
tt

[0, 20]

In [377]:
submit

[['A10', 'B12', 'B15', 'B7'],
 ['A12', 'B4', 'B5', 'B8'],
 ['A13', 'B19', 'B6', 'B7'],
 ['A15', 'B14', 'B19', 'B8'],
 ['A3', 'B1', 'B4', 'B5'],
 ['A4', 'B10', 'B17', 'B2'],
 ['A5', 'B12', 'B16', 'B5'],
 ['A6', 'B13', 'B18', 'B5'],
 ['A7', 'B14', 'B17', 'B19'],
 ['A8', 'B15', 'B18', 'B7'],
 ['A15', 'B13', 'B14', 'B16'],
 ['A14', 'B17', 'B18', 'B3'],
 ['A13', 'B17', 'B6', 'B8'],
 ['A12', 'B13', 'B14', 'B19'],
 ['A11', 'B14', 'B19', 'B5'],
 ['A10', 'B13', 'B19', 'B9'],
 ['A9', 'B12', 'B15', 'B4'],
 ['A8', 'B10', 'B15', 'B7'],
 ['A7', 'B17', 'B8', 'B9'],
 ['A6', 'B10', 'B14', 'B6'],
 ['A5', 'B11', 'B13', 'B19'],
 ['A4', 'B2', 'B5', 'B6'],
 ['A3', 'B10', 'B15', 'B9'],
 ['A2', 'B12', 'B14', 'B15'],
 ['A1', 'B11', 'B12', 'B4'],
 ['A15', 'B16', 'B2', 'B3'],
 ['A14', 'B10', 'B3', 'B9'],
 ['A13', 'B1', 'B12', 'B3'],
 ['A12', 'B11', 'B17', 'B9'],
 ['A11', 'B17', 'B4', 'B5'],
 ['A10', 'B1', 'B19', 'B9'],
 ['A9', 'B1', 'B19', 'B6'],
 ['A8', 'B1', 'B17', 'B2'],
 ['A7', 'B10', 'B4', 'B9'],
 ['A6', 'B

In [382]:
B

['B18',
 'B18',
 'B18',
 'B1',
 'B8',
 'B9',
 'B10',
 'B11',
 'B12',
 'B16',
 'B18',
 'B8',
 'B7',
 'B6',
 'B5',
 'B19',
 'B16',
 'B13',
 'B12',
 'B10']

# C組牌 配率
>每包內含張數：3
>同一包卡片不重複	

In [378]:
data_C = data[data['組合'] == 'C']
data_C = data_C.reset_index(drop=True)
data_C

,組合,組合占比,卡片名稱,卡片種類,卡片總量
0,C,50%,勝利條件卡N,6,150000
1,C,50%,勝利條件卡SP,10,150000
